# Abrestnet code file starts here

In [1]:
import argparse
import os
from tqdm import tqdm
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter

import sys
sys.path.append('/home/pasang/all_experiment/FvFold')

import fvfold
from fvfold.models.AbResNet import AbResNet, load_model

from fvfold.util.masking import MASK_VALUE
from fvfold.util.training import check_for_h5_file
# from fvfold.util.util import RawTextArgumentDefaultsHelpFormatter
from fvfold.datasets.H5PairwiseGeometryDataset import H5PairwiseGeometryDataset
from fvfold.preprocess.generate_h5_pairwise_geom_file import antibody_to_h5

_output_names = ['ca_dist', 'cb_dist', 'no_dist', 'omega', 'theta', 'phi']
os.getcwd()

/home/pasang/anaconda3/envs/antibody/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/pasang/all_experiment/FvFold/fvfold/models/AbResNet'

In [2]:
import torch

if torch.cuda.is_available():
    print("GPUs available:")
    for i in range(torch.cuda.device_count()):
        device_properties = torch.cuda.get_device_properties(i)
        print(f"{i}: {device_properties.name}, Memory: {device_properties.total_memory / (1024**2):.2f} MB")
else:
    print("No GPUs available.")

GPUs available:
0: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
1: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
2: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
3: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
4: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
5: NVIDIA GeForce RTX 2080 Ti, Memory: 11019.56 MB
6: NVIDIA RTX A6000, Memory: 48685.31 MB
7: NVIDIA A100 80GB PCIe, Memory: 81069.75 MB


In [3]:
project_path = os.path.abspath(os.path.join(fvfold.__file__, "../.."))
num_blocks1D=3
num_blocks2D=25
dilation_cycle=5
num_bins=37
dropout=0.2
num_blocks2D=25
dilation_cycle=5
num_bins=37
dropout=0.2
epochs=200
save_every=5
batch_size=8
lr=0.01
use_gpu=True
train_split=0.9
h5_file= os.path.join(project_path, 'data/final_antibody.h5')
antibody_database=os.path.join(project_path,'data/antibody_database')
now = str(datetime.now().strftime('%y-%m-%d %H:%M:%S'))
default_model_path = os.path.join(project_path,'trained_models/model_{}/'.format(now))
output_dir=default_model_path
pretrain_model_file=None
random_seed=0
project_path


'/home/pasang/all_experiment/FvFold'

In [4]:
device_type = 'cuda:7' if torch.cuda.is_available(
    ) and use_gpu else 'cpu'
device = torch.device(device_type)
out_dir = output_dir
current_epoch = 0

In [5]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self,
                    weight=None,
                    gamma=2,
                    reduction='mean',
                    ignore_index=MASK_VALUE):
        super(FocalLoss, self).__init__(weight, reduction=reduction)
        self.gamma = gamma
        self.weight = weight
        self.ignore_index = ignore_index

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input,
                                    target,
                                    reduction=self.reduction,
                                    weight=self.weight,
                                    ignore_index=self.ignore_index)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt)**self.gamma * ce_loss).mean()
        return focal_loss

In [6]:
# properties of resnet and 2d resnet
properties = dict(num_out_bins=num_bins,
                    num_blocks1D=num_blocks1D,
                    num_blocks2D=num_blocks2D,
                    dropout_proportion=dropout,
                    dilation_cycle=dilation_cycle)
print("abresnet properties",properties)

abresnet properties {'num_out_bins': 37, 'num_blocks1D': 3, 'num_blocks2D': 25, 'dropout_proportion': 0.2, 'dilation_cycle': 5}


In [7]:
model = AbResNet(21, **properties)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
properties.update({'lr': lr})
print("properties",properties)
print("model",model)

properties {'num_out_bins': 37, 'num_blocks1D': 3, 'num_blocks2D': 25, 'dropout_proportion': 0.2, 'dilation_cycle': 5, 'lr': 0.01}
model AbResNet(
  (linear): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (resnet1D): ResNet1D(
    (conv1): Conv1d(21, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer0): Sequential(
      (0): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32

In [8]:
print('Making {} ...'.format(out_dir))
os.mkdir(out_dir)

print(properties)

Making /home/pasang/all_experiment/FvFold/trained_models/model_23-12-26 16:00:10/ ...
{'num_out_bins': 37, 'num_blocks1D': 3, 'num_blocks2D': 25, 'dropout_proportion': 0.2, 'dilation_cycle': 5, 'lr': 0.01}


# Load only train loader not validation for testing purpose

In [9]:
properties.update({'lr': lr})

check_for_h5_file(h5_file, antibody_to_h5, antibody_database)
dataset = H5PairwiseGeometryDataset(h5_file,
                                    num_bins=num_bins,
                                    mask_distant_orientations=True)
train_split_length = int(len(dataset) * train_split)
torch.manual_seed(random_seed)
train_dataset, validation_dataset = data.random_split(
    dataset, [train_split_length,
                len(dataset) - train_split_length])
train_loader = data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=H5PairwiseGeometryDataset.merge_samples_to_minibatch)
validation_loader = data.DataLoader(
    validation_dataset,
    batch_size=batch_size,
    collate_fn=H5PairwiseGeometryDataset.merge_samples_to_minibatch)
len(dataset)

22

In [10]:
criterion = FocalLoss(ignore_index=dataset.mask_fill_value)

lr_modifier = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                            verbose=True)
writer = SummaryWriter(os.path.join(out_dir, 'tensorboard'))




print('Model:\n', model)

Model:
 AbResNet(
  (linear): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (resnet1D): ResNet1D(
    (conv1): Conv1d(21, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer0): Sequential(
      (0): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(32, 32, kernel_size=(17,), stride=(1,),

In [11]:
model=model
train_loader=train_loader
# validation_loader=validation_loader
optimizer=optimizer
device=device
epochs=epochs
current_epoch=current_epoch
criterion=criterion
lr_modifier=lr_modifier
writer=writer
save_file=os.path.join(out_dir, 'model')
save_every=save_every
properties=properties
properties

{'num_out_bins': 37,
 'num_blocks1D': 3,
 'num_blocks2D': 25,
 'dropout_proportion': 0.2,
 'dilation_cycle': 5,
 'lr': 0.01}

In [12]:
properties = {} if properties is None else properties
print('Using {} as device'.format(str(device).upper()))
model = model.to(device)
print(model)
loss_size = len(_output_names) + 1

Using CUDA:7 as device
AbResNet(
  (linear): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (resnet1D): ResNet1D(
    (conv1): Conv1d(21, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer0): Sequential(
      (0): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResBlock1D(
        (conv1): Conv1d(32, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(32, 32, kernel_size=(17,

In [13]:
def train_epoch(model, train_loader, optimizer, device, criterion, loss_size):
    """Trains a model for one epoch"""
    model.train()
    running_losses = torch.zeros(loss_size)
    for inputs,e,max_len,labels in tqdm(train_loader, total=len(train_loader)):
        inputs = inputs.to(device)
        labels = [label.to(device) for label in labels]
        
        emb=torch.stack([x.clone().detach() for x in e])
        tensor_list = [torch.tensor(x, dtype=torch.int32) for x in max_len]
        unpadded_tensor = emb[:, :, :max(tensor_list)]
        

        optimizer.zero_grad()

        def handle_batch():
            outputs = model(inputs,unpadded_tensor.to(device))

            losses = [
                criterion(output, label)
                for output, label in zip(outputs, labels)
            ]
            total_loss = sum(losses)
            losses.append(total_loss)

            total_loss.backward()
            optimizer.step()
            return outputs, torch.Tensor([float(l.item()) for l in losses])

        outputs, batch_loss = handle_batch()
        running_losses += batch_loss

    return running_losses


def validate(model, validation_loader, device, criterion, loss_size):
    """"""
    with torch.no_grad():
        model.eval()
        running_losses = torch.zeros(loss_size)
        for  inputs,e,max_len,labels in tqdm(validation_loader,
                                   total=len(validation_loader)):
            inputs = inputs.to(device)
            labels = [label.to(device) for label in labels]
            emb=torch.stack([x.clone().detach() for x in e])
            tensor_list = [torch.tensor(x, dtype=torch.int32) for x in max_len]
            unpadded_tensor = emb[:, :, :max(tensor_list)]

            def handle_batch():
                outputs = model(inputs,unpadded_tensor.to(device))

                losses = [
                    criterion(output, label)
                    for output, label in zip(outputs, labels)
                ]
                total_loss = sum(losses)
                losses.append(total_loss)

                return outputs, torch.Tensor([float(l.item()) for l in losses])

            outputs, batch_loss = handle_batch()
            running_losses += batch_loss
    return running_losses

In [14]:
use_loss=[]
avg_v_loss=[]
avg_t_loss=[]
early_stop = False
best_val_loss = float('inf')
patience = 40
counter = 0

for epoch in range(current_epoch, epochs):
    train_losses = train_epoch(model, train_loader, optimizer, device, criterion, loss_size)
    avg_train_losses = train_losses / len(train_loader)

    use_loss.append(avg_train_losses[6])
    train_loss_dict = dict(zip(_output_names + ['total'], avg_train_losses.tolist()))
    writer.add_scalars('train_loss', train_loss_dict, global_step=epoch)
    print('\nAverage training loss (epoch {}): {}'.format(epoch, train_loss_dict))

    val_losses = validate(model, validation_loader, device, criterion, loss_size)
    avg_val_losses = val_losses / len(validation_loader)
    val_loss_dict = dict(zip(_output_names + ['total'], avg_val_losses.tolist()))
    writer.add_scalars('validation_loss', val_loss_dict, global_step=epoch)
    print('\nAverage validation loss (epoch {}): {}'.format(epoch, val_loss_dict))

    total_val_loss = val_losses[-1]
    lr_modifier.step(total_val_loss)

    avg_v_loss.append(val_loss_dict['total'])
    avg_t_loss.append(train_loss_dict['total'])

    if val_loss_dict['total'] < best_val_loss:
        best_val_loss = val_loss_dict['total']
        counter = 0
        properties.update({'model_state_dict': model.state_dict()})
        properties.update({'optimizer_state_dict': optimizer.state_dict(), 'train_loss': train_loss_dict, 'val_loss': val_loss_dict, 'epoch': epoch})
        torch.save(properties, save_file + "_rs{}.pt".format(epoch))
    else:
        counter += 1
        if counter >= patience:
            early_stop = True
            break

    if early_stop:
        break
    
    torch.cuda.empty_cache()


properties.update({'model_state_dict': model.state_dict()})
properties.update({
    'optimizer_state_dict': optimizer.state_dict(),
    'train_loss': train_loss_dict,
    'val_loss': val_loss_dict,
    'epoch': epoch
})
torch.save(properties, save_file+".pt")

100%|██████████| 3/3 [00:18<00:00,  6.32s/it]



Average training loss (epoch 0): {'ca_dist': 3.7945549488067627, 'cb_dist': 3.77087140083313, 'no_dist': 3.566787004470825, 'omega': 3.743830919265747, 'theta': 3.485022783279419, 'phi': 3.548717737197876, 'total': 21.9097843170166}


  0%|          | 0/1 [00:00<?, ?it/s]/home/pasang/.local/lib/python3.7/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 1/1 [00:00<00:00,  1.24it/s]



Average validation loss (epoch 0): {'ca_dist': 1562516127744.0, 'cb_dist': 205191823360.0, 'no_dist': 1562351370240.0, 'omega': 539834941440.0, 'theta': 49077133312.0, 'phi': 175577612288.0, 'total': 4094548770816.0}


100%|██████████| 3/3 [00:14<00:00,  4.73s/it]


...


Average training loss (epoch 24): {'ca_dist': 1.2994036674499512, 'cb_dist': 1.3041633367538452, 'no_dist': 2.0690016746520996, 'omega': 1.2811778783798218, 'theta': 2.045006275177002, 'phi': 2.0588009357452393, 'total': 10.057554244995117}


100%|██████████| 1/1 [00:00<00:00,  1.13it/s]



Average validation loss (epoch 24): {'ca_dist': 1.1692897081375122, 'cb_dist': 1.573205828666687, 'no_dist': 1.8018957376480103, 'omega': 1.2140069007873535, 'theta': 2.0696465969085693, 'phi': 1.9836407899856567, 'total': 9.811685562133789}


  0%|          | 0/3 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:

#t=0.42 v=0.56
import matplotlib.pyplot as plt
plt.plot(use_loss)
plt.ylabel('Loss $L(y,\hat{y};a)$')
plt.xlabel('Epochs')
print(use_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(avg_t_loss)
print(type(avg_t_loss))
plt.ylabel('Loss $L(y,\hat{y};a)$')
print(avg_t_loss)
plt.xlabel('Epochs')
# print(avg_t_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(avg_v_loss)
plt.ylabel('Loss $L(y,\hat{y};a)$')
plt.xlabel('Epochs')
print(avg_v_loss)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Validation Loss")
plt.plot(avg_v_loss,label="val")
plt.plot(avg_t_loss,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()